<a href="https://colab.research.google.com/github/alelasantillan/gpt2-example/blob/master/First_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Analyticsvidhya - GPT-2 implementation

In [7]:
!git clone https://github.com/openai/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 188 (delta 0), reused 0 (delta 0), pack-reused 184
Receiving objects: 100% (188/188), 4.36 MiB | 3.85 MiB/s, done.
Resolving deltas: 100% (94/94), done.


In [0]:
import os
os.chdir("gpt-2")


In [4]:
os.getcwd()

'/content/gpt-2'

In [9]:
os.listdir()

['requirements.txt',
 'download_model.py',
 'src',
 'LICENSE',
 'Dockerfile.gpu',
 '.gitattributes',
 '.git',
 'CONTRIBUTORS.md',
 '.gitignore',
 'DEVELOPERS.md',
 'README.md',
 'Dockerfile.cpu']

In [6]:
!pip3 install tensorflow-gpu==1.12.0

In [7]:
!pip3 install -r requirements.txt

In [9]:
!python3 download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 564kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 45.3Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 547kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:22, 62.2Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 5.95Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 44.4Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 44.7Mit/s]                                                       


In [0]:
!export PYTHONIOENCODING=UTF-8

In [0]:
#os.chdir("gpt-2")
os.chdir('src')

In [17]:
os.listdir()

['encoder.py',
 'sample.py',
 'interactive_conditional_samples.py',
 '__pycache__',
 'model.py',
 'generate_unconditional_samples.py']

In [16]:
os.getcwd()

'/content/gpt-2/src'

In [26]:
!pip3 install regex

     |████████████████████████████████| 655kB 40.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/35/e4/80/abf3b33ba89cf65cd262af8a22a5a999cc28fbfabea6b38473
Successfully built regex


In [0]:
import json
import os
import numpy as np
import tensorflow as tf
import sys

file_dir = os.path.dirname("/content/gpt-2/src")
sys.path.append(file_dir)
import model, sample, encoder


In [0]:
def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)